# Real time area light

## Light transport equation (LTE in abbrev.)

Firstly, let's look at the LTE
$$L_o(W_o, P) = L_e(W_o, P) + \int{L_i(W_i, P)}{f_r(W_o, W_i)}\overline{\langle W_o, W_i \rangle} d{W_i}$$
It's a simplified version of radiative transfer equation which involve integral differential equation. Let's make our life easier and focus on LTE.

As we can see in following picture, $L_o$ and $L_i$ term is interchangeable in following relationship.
$$L_i(W_i, P) = L_o(-W_i, T(P, W_i)), where \space T(P, \vec{W)} = P + t * \vec{W}$$

Plugin this equation into LTE and simplify the notation, we have
$$L(W, P) = L_e(W, P) + \int{L(-W_i, T(P, W_i))}{f_r(W, W_i)}\overline{\langle W, W_i \rangle} d{W_i}$$

## Path space integration

## Analytic solution for diffusion area light on lambert shading model

* Sphere light
* Disk light
* Rectangle light
* Polygonal light

In [ ]:
import numpy as np

r = 5
P = np.array([10, 10])
theta_0 = np.arctan2(P[1], P[0])
delta_theta = np.arctan2(r, np.linalg.norm(P))
result = -np.cos(theta_0 + delta_theta) + np.cos(theta_0 - delta_theta)
print("theta 1 = ", theta_0 - delta_theta)
print("theta 2 = ", theta_0 + delta_theta)

print("Analytic result = ", result)

mc_result = 0
dTheta = 0.001
for theta in np.arange(0, np.pi, dTheta):
    cos_theta = np.sin(theta)
    if theta >= theta_0 - delta_theta and theta <= theta_0 + delta_theta:
        mc_result += cos_theta
mc_result *= dTheta
print("Monte carlo result = ", mc_result)

## Area light on microfacet shading model with linearly transform cosine

## Area light on microfacet shading model with spherical harmonics

## Real-time monte carlo integration with denoiser

## Future work

* Volume rendering
* 

## References

* [RADIATIVE VIEW FACTORS](http://webserver.dmt.upm.es/~isidoro/tc3/Radiation%20View%20factors.pdf)
* [sphere ambient occlusion](http://www.iquilezles.org/www/articles/sphereao/sphereao.htm)